# Lecture 5: Bayes' rule

In [1]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## Part 1: Simulating cancer testing 

In [2]:
set.seed(22)

In [3]:
num_patients <- 1e6
incidence <- 0.009
tpr <- 0.75
tnr <- 0.9

In [4]:
patients <- tibble(
    cancer = rbernoulli(num_patients, incidence),
    test = rbernoulli(num_patients, if_else(cancer, tpr, 1-tnr))
)

In [5]:
head(patients)

cancer,test
<lgl>,<lgl>
FALSE,FALSE
FALSE,TRUE
TRUE,TRUE
FALSE,FALSE
FALSE,FALSE
FALSE,FALSE


In [6]:
patients %>%
    filter(test) %>%
    summarize(p = mean(cancer))

p
<dbl>
0.06453958
